
🎓 Author

Amidou MAIGA
MSc Student – Energy and Renewable Energy
Department of Energy Systems Engineering
Erciyes University, Kayseri, Türkiye
MAil : amidou.maiga@2ie-edu.com
LinkedIn : www.linkedin.com/in/amidou-maiga-etude


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import matplotlib
matplotlib.use('TkAgg')

class PEMFC_DynamicModel:
    """Dynamic PEMFC model with Tkinter interface"""
    
    def __init__(self):
        # Physical constants
        self.F = 96485.3329      # C/mol
        self.R = 8.314462618     # J/(mol·K)
        
        # Default parameters
        self.default_params = {
            'T_K': 353.15,        # K (303.15 to 353.15 K = 30 to 80°C)
            'P_H2': 1.5,          # atm (1.0 to 3.0)
            'P_O2': 1.0,          # atm (1.0 to 3.0)
            'RH': 80,             # %
            'i0': 1e-6,           # A/cm² (default 10⁻⁶)
            'alpha': 0.5,         # transfer coefficient (≈0.5)
            'i_lim': 2.0,         # A/cm²
            'R_ohm': 0.1,         # Ω·cm²
            'B': 0.05,            # V, concentration coefficient
            'membrane': 'Nafion 211'
        }
        
        # Membranes properties
        self.membrane_props = {
            'Nafion 211': {'thickness': 25, 'conductivity': 0.12},    # 25 μm
            'Nafion 112': {'thickness': 51, 'conductivity': 0.10},    # 51 μm
            'Nafion 117': {'thickness': 183, 'conductivity': 0.08}    # 183 μm
        }
        
        self.root = None
        self.setup_ui()
        
        print("✅ PEMFC_DynamicModel initialisé")
    
    # ========== CALCULATION METHODS ==========
    
    def calculate_E_nernst(self, T_K, P_H2, P_O2):
        """Calculate Nernst potential"""
        # Avoid log(0)
        P_H2_safe = max(P_H2, 1e-10)
        P_O2_safe = max(P_O2, 1e-10)
        
        return (1.229 - 0.85e-3*(T_K-298.15) + 
                4.3085e-5*T_K*np.log(P_H2_safe*np.sqrt(P_O2_safe)))
    
    def calculate_eta_act(self, i, T_K, i0, alpha):
        """Activation losses (Tafel equation)"""
        # Avoid log(0)
        i_safe = np.maximum(i, i0*1.001)
        beta = self.R*T_K/(alpha*self.F)
        return beta * np.log(i_safe/i0)
    
    def calculate_eta_ohm(self, i, R_ohm, T_K, membrane_type):
        """Ohmic losses with temperature dependence"""
        # Conductivity depends on temperature
        props = self.membrane_props[membrane_type]
        sigma = props['conductivity'] * np.exp(0.034*(T_K-298.15))
        R_mem = props['thickness'] * 1e-4 / max(sigma, 1e-6)  # Convert μm to cm
        return i * (R_ohm + R_mem)
    
    def calculate_eta_conc(self, i, i_lim, B):
        """Concentration losses"""
        i_safe = np.minimum(i, i_lim*0.999)
        return B * np.log(i_lim/(i_lim - i_safe))
    
    def calculate_curve(self, params=None):
        """Calculate polarization curve"""
        if params is None:
            params = self.default_params
        
        T_K = params['T_K']
        P_H2 = params['P_H2']
        P_O2 = params['P_O2']
        
        # Current range (up to 95% of limiting current)
        i_max = min(params['i_lim'] * 0.95, 3.0)
        i = np.linspace(0.001, i_max, 200)
        
        # Calculations
        E_rev = self.calculate_E_nernst(T_K, P_H2, P_O2)
        eta_act = self.calculate_eta_act(i, T_K, params['i0'], params['alpha'])
        eta_ohm = self.calculate_eta_ohm(i, params['R_ohm'], T_K, params['membrane'])
        eta_conc = self.calculate_eta_conc(i, params['i_lim'], params['B'])
        
        # Voltage and power
        V = E_rev - eta_act - eta_ohm - eta_conc
        V = np.maximum(V, 0)  # Avoid negative values
        P = V * i  # W/cm²
        
        return i, V, P, E_rev, eta_act, eta_ohm, eta_conc
    
    # ========== UI METHODS ==========
    
    def get_params(self):
        """Get parameters from interface"""
        params = {
            'T_K': float(self.temp_slider.get()),
            'P_H2': float(self.ph2_slider.get()),
            'P_O2': float(self.po2_slider.get()),
            'RH': float(self.rh_slider.get()),
            'i0': float(self.i0_var.get()),
            'alpha': float(self.alpha_slider.get()),
            'i_lim': float(self.ilim_slider.get()),
            'R_ohm': float(self.rohm_slider.get()),
            'B': float(self.b_slider.get()),
            'membrane': self.mem_var.get()
        }
        return params
    
    def update_plot(self):
        """Update all graphs"""
        try:
            # Get parameters
            params = self.get_params()
            
            # Calculate curve
            i, V, P, E_rev, eta_act, eta_ohm, eta_conc = self.calculate_curve(params)
            
            # === CLEAR GRAPHS ===
            for ax in [self.ax1, self.ax2, self.ax3, self.ax4, self.ax5]:
                ax.clear()
                if ax != self.ax5:
                    ax.grid(True, alpha=0.2, linestyle='--')
                    ax.set_facecolor('#ffffff')
            
            # === 1. POLARIZATION CURVE ===
            self.ax1.plot(i, V, 'b-', linewidth=2.5, label='V(i) curve')
            self.ax1.axhline(y=E_rev, color='r', linestyle='--', linewidth=1.5, 
                           alpha=0.7, label=f'E_rev = {E_rev:.3f} V')
            
            # Activation zone
            activation_mask = i < 0.3
            if np.any(activation_mask):
                self.ax1.fill_between(i[activation_mask], 0, V[activation_mask], 
                                     alpha=0.2, color='red', label='Activation zone')
            
            # Ohmic zone
            ohm_mask = (i >= 0.3) & (i < 1.5)
            if np.any(ohm_mask):
                self.ax1.fill_between(i[ohm_mask], 0, V[ohm_mask], 
                                     alpha=0.2, color='blue', label='Ohmic zone')
            
            # Concentration zone
            
            conc_mask = i >= 1.5
            if np.any(conc_mask):
                self.ax1.fill_between(i[conc_mask], 0, V[conc_mask], 
                                     alpha=0.2, color='green', label='Concentration zone')
            
            self.ax1.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
            self.ax1.set_ylabel('Cell Voltage (V)', fontsize=11, fontweight='bold')
            self.ax1.set_title('PEMFC Polarization Curve', fontsize=12, fontweight='bold', pad=15)
            self.ax1.legend(loc='upper right', fontsize=9, framealpha=0.9)
            self.ax1.set_xlim(0, np.max(i))
            self.ax1.set_ylim(0, max(V)*1.1)
            
            # === 2. POWER CURVE ===
            self.ax2.plot(i, P, 'g-', linewidth=2.5, label='Power')
            
            # Maximum power point
            max_power_idx = np.argmax(P)
            self.ax2.plot(i[max_power_idx], P[max_power_idx], 'ro', 
                         markersize=10, markeredgecolor='black', 
                         label=f'P_max = {P[max_power_idx]:.3f} W/cm²')
            
            # Vertical line at max power point
            self.ax2.axvline(x=i[max_power_idx], color='r', linestyle=':', alpha=0.5)
            
            self.ax2.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
            self.ax2.set_ylabel('Power (W/cm²)', fontsize=11, fontweight='bold')
            self.ax2.set_title('Power Density', fontsize=12, fontweight='bold', pad=15)
            self.ax2.legend(loc='upper right', fontsize=9, framealpha=0.9)
            self.ax2.set_xlim(0, np.max(i))
            
            # === 3. LOSS DISTRIBUTION ===
            self.ax3.plot(i, eta_act, 'r-', linewidth=2, label='η_activation')
            self.ax3.plot(i, eta_ohm, 'b-', linewidth=2, label='η_ohmic')
            self.ax3.plot(i, eta_conc, 'g-', linewidth=2, label='η_concentration')
            
            # Total losses
            eta_total = eta_act + eta_ohm + eta_conc
            self.ax3.plot(i, eta_total, 'k--', linewidth=1.5, label='η_total')
            
            self.ax3.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
            self.ax3.set_ylabel('Overpotential (V)', fontsize=11, fontweight='bold')
            self.ax3.set_title('Loss Distribution', fontsize=12, fontweight='bold', pad=15)
            self.ax3.legend(loc='upper left', fontsize=9, framealpha=0.9)
            self.ax3.set_xlim(0, np.max(i))
            
            # === 4. EFFICIENCY ===
            efficiency = (V / E_rev) * 100
            self.ax4.plot(i, efficiency, 'purple', linewidth=2.5, label='Efficiency')
            
            # Efficiency at max power point
            eff_at_pmax = efficiency[max_power_idx]
            self.ax4.plot(i[max_power_idx], eff_at_pmax, 'ro', 
                         markersize=8, markeredgecolor='black',
                         label=f'Efficiency @ P_max = {eff_at_pmax:.1f}%')
            
            self.ax4.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
            self.ax4.set_ylabel('Efficiency (%)', fontsize=11, fontweight='bold')
            self.ax4.set_title('Voltage Efficiency', fontsize=12, fontweight='bold', pad=15)
            self.ax4.legend(loc='upper right', fontsize=9, framealpha=0.9)
            self.ax4.set_xlim(0, np.max(i))
            self.ax4.set_ylim(0, 100)

            # === 5. INFORMATION PANEL ===
            self.ax5.axis('off')
            
            # Get membrane properties
            mem_props = self.membrane_props[params['membrane']]
            
            # Calculated information
            info_text = f"""INPUT PARAMETERS:


🌡️ Temperature: {params['T_K']:.2f} K ({params['T_K']-273.15:.1f}°C)
⚡ Anode Pressure (H₂): {params['P_H2']:.1f} atm
💨 Cathode Pressure (O₂): {params['P_O2']:.1f} atm
💧 Relative Humidity: {params['RH']:.0f}%

ELECTROCHEMICAL PARAMETERS:
🔬 Exchange current density (i₀): {params['i0']:.1e} A/cm²
📏 Transfer coefficient (α): {params['alpha']:.2f}
⚡ Limiting current (i_L): {params['i_lim']:.2f} A/cm²
🔋 Ohmic resistance: {params['R_ohm']:.3f} Ω·cm²
📈 Concentration coefficient (B): {params['B']:.3f} V

MATERIAL PROPERTIES:
🔧 Membrane: {params['membrane']}
📏 Thickness: {mem_props['thickness']} μm
⚡ Conductivity: {mem_props['conductivity']} S/cm

PERFORMANCE RESULTS:
🎯 Reversible potential: {E_rev:.3f} V
💪 Maximum power: {P[max_power_idx]:.4f} W/cm²
🔋 Current @ P_max: {i[max_power_idx]:.3f} A/cm²
📈 Voltage @ P_max: {V[max_power_idx]:.3f} V
🎭 Efficiency @ P_max: {eff_at_pmax:.1f}%"""
            
            self.ax5.text(0.02, 0.98, info_text, fontsize=6,
                         verticalalignment='top',
                         bbox=dict(boxstyle="round,pad=0.5",
                                 facecolor="lightyellow",
                                 edgecolor="gold",
                                 alpha=0.9))
            
            # Update canvas
            self.fig.tight_layout()
            self.canvas.draw()
            
            # Update status
            self.status_label.config(
                text=f"✅ Curve calculated - P_max = {P[max_power_idx]:.3f} W/cm² - " +
                     f"Efficiency = {eff_at_pmax:.1f}%", 
                fg="#27ae60"
            )
            
        except Exception as e:
            self.status_label.config(
                text=f"❌ Error: {str(e)}", 
                fg="#e74c3c"
            )
            messagebox.showerror("Calculation Error", f"An error occurred:\n{str(e)}")
    
    def export_plot(self):
        """Export plots """
        try:
            # Ask for filename
            filename = filedialog.asksaveasfilename(
                defaultextension=".png",
                filetypes=[
                    ("PNG files", "*.png"),
                    ("PDF files", "*.pdf"),
                    ("SVG files", "*.svg"),
                    ("All files", "*.*")
                ],
                initialfile="pemfc_polarization_curve.png"
            )
            
            if filename:
                # Adjust resolution for publication quality
                self.fig.savefig(filename, dpi=300, bbox_inches='tight', 
                               facecolor=self.fig.get_facecolor())
                
                messagebox.showinfo("Export Successful", 
                                  f"Plot exported successfully:\n{filename}")
                self.status_label.config(
                    text=f"✅ Plot exported", 
                    fg="#2980b9"
                )
                
        except Exception as e:
            messagebox.showerror("Export Error", f"Cannot export:\n{str(e)}")
    
    def export_data(self):
        """Export numerical data"""
        try:
            # Get current data
            params = self.get_params()
            i, V, P, E_rev, eta_act, eta_ohm, eta_conc = self.calculate_curve(params)
            
            # Ask for filename
            filename = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[
                    ("CSV files", "*.csv"),
                    ("Text files", "*.txt"),
                    ("All files", "*.*")
                ],
                initialfile="pemfc_simulation_data.csv"
            )
            
            if filename:
                # Create data
                data = []
                data.append("# PEMFC Simulation Data")
                data.append(f"# Temperature: {params['T_K']} K ({params['T_K']-273.15:.1f}°C)")
                data.append(f"# Anode Pressure H2: {params['P_H2']} atm")
                data.append(f"# Cathode Pressure O2: {params['P_O2']} atm")
                data.append(f"# Relative Humidity: {params['RH']} %")
                data.append(f"# Membrane: {params['membrane']}")
                data.append(f"# Exchange current density i₀: {params['i0']} A/cm²")
                data.append(f"# Transfer coefficient α: {params['alpha']}")
                data.append(f"# Limiting current i_L: {params['i_lim']} A/cm²")
                data.append(f"# Ohmic resistance: {params['R_ohm']} Ω·cm²")
                data.append(f"# Concentration coefficient B: {params['B']} V")
                data.append("#")
                data.append("i(A/cm2),V(V),P(W/cm2),eta_act(V),eta_ohm(V),eta_conc(V),Efficiency(%)")
                
                for idx in range(len(i)):
                    efficiency = (V[idx] / E_rev) * 100 if E_rev > 0 else 0
                    data.append(f"{i[idx]:.6f},{V[idx]:.6f},{P[idx]:.6f},{eta_act[idx]:.6f},{eta_ohm[idx]:.6f},{eta_conc[idx]:.6f},{efficiency:.2f}")
                
                # Write file
                with open(filename, 'w', encoding="utf-8") as f:
                    f.write('\n'.join(data))
                
                messagebox.showinfo("Export Successful", 
                                  f"Data exported successfully:\n{filename}\n\n" +
                                  f"Data points: {len(i)}")
                self.status_label.config(
                    text=f"✅ Data exported: {len(i)} points", 
                    fg="#2980b9"
                )
                
        except Exception as e:
            messagebox.showerror("Export Error", f"Cannot export data:\n{str(e)}")
    
    def reset_params(self):
        """Reset to default parameters"""
        self.temp_slider.set(self.default_params['T_K'])
        self.ph2_slider.set(self.default_params['P_H2'])
        self.po2_slider.set(self.default_params['P_O2'])
        self.rh_slider.set(self.default_params['RH'])
        self.i0_var.set("1e-6")
        self.alpha_slider.set(self.default_params['alpha'])
        self.ilim_slider.set(self.default_params['i_lim'])
        self.rohm_slider.set(self.default_params['R_ohm'])
        self.b_slider.set(self.default_params['B'])
        self.mem_var.set(self.default_params['membrane'])
        
        # Update labels
        self.temp_value.set(f"{self.default_params['T_K']:.1f} K")
        self.ph2_value.set(f"{self.default_params['P_H2']} atm")
        self.po2_value.set(f"{self.default_params['P_O2']} atm")
        self.rh_value.set(f"{self.default_params['RH']}%")
        self.alpha_value.set(f"{self.default_params['alpha']:.2f}")
        self.ilim_value.set(f"{self.default_params['i_lim']:.1f}")
        self.rohm_value.set(f"{self.default_params['R_ohm']:.2f}")
        self.b_value.set(f"{self.default_params['B']:.3f}")
        
        self.status_label.config(text="🔄 Parameters reset", fg="#f39c12")
        self.update_plot()
    
    def show_analysis(self):
        """Show analysis window"""
        analysis_window = tk.Toplevel(self.root)
        analysis_window.title("Performance Analysis")
        analysis_window.geometry("600x400")
        
        text = tk.Text(analysis_window, wrap=tk.WORD, font=("Arial", 10),
                      bg="#f8f9fa", padx=10, pady=10)
        scrollbar = ttk.Scrollbar(analysis_window, command=text.yview)
        text.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        text.pack(fill=tk.BOTH, expand=True)
        
        # Get current data for analysis
        params = self.get_params()
        i, V, P, E_rev, eta_act, eta_ohm, eta_conc = self.calculate_curve(params)
        max_power_idx = np.argmax(P)
        
        analysis_text = f"""
        PERFORMANCE ANALYSIS
        ====================
        
        OPERATING CONDITIONS:
        • Temperature: {params['T_K']} K ({params['T_K']-273.15:.1f}°C)
        • Anode Pressure (H2): {params['P_H2']} atm
        • Cathode Pressure (O2): {params['P_O2']} atm
        • Relative Humidity: {params['RH']} %
        
        ELECTROCHEMICAL PARAMETERS:
        • Exchange current density (i₀): {params['i0']:.1e} A/cm²
        • Transfer coefficient (α): {params['alpha']:.2f}
        • Limiting current (i_L): {params['i_lim']} A/cm²
        • Ohmic resistance: {params['R_ohm']} Ω·cm²
        
        MATERIAL PROPERTIES:
        • Membrane: {params['membrane']}
        • Thickness: {self.membrane_props[params['membrane']]['thickness']} μm
        • Conductivity: {self.membrane_props[params['membrane']]['conductivity']} S/cm
        
        KEY PERFORMANCE INDICATORS:
        • Maximum Power: {P[max_power_idx]:.4f} W/cm²
        • Current at P_max: {i[max_power_idx]:.3f} A/cm²
        • Voltage at P_max: {V[max_power_idx]:.3f} V
        • Efficiency at P_max: {(V[max_power_idx]/E_rev)*100:.1f} %
        
        LOSS ANALYSIS:
        • Activation Loss @ P_max: {eta_act[max_power_idx]:.3f} V
        • Ohmic Loss @ P_max: {eta_ohm[max_power_idx]:.3f} V
        • Concentration Loss @ P_max: {eta_conc[max_power_idx]:.3f} V
        • Total Loss @ P_max: {eta_act[max_power_idx] + eta_ohm[max_power_idx] + eta_conc[max_power_idx]:.3f} V
        
        RECOMMENDATIONS:
        1. To increase power: Raise temperature or pressure
        2. To reduce activation loss: Increase i₀ value
        3. To reduce ohmic loss: Use thinner membrane (Nafion 211)
        4. To extend operating range: Increase i_L value
        """
        
        text.insert(tk.END, analysis_text)
        text.config(state=tk.DISABLED)
        
        ttk.Button(analysis_window, text="Close", command=analysis_window.destroy).pack(pady=10)
    
    def show_equations(self):
        """Show model equations"""
        eq_window = tk.Toplevel(self.root)
        eq_window.title("PEMFC Model Equations")
        eq_window.geometry("800x600")
        
        # Text with equations
        eq_text = """
        PEMFC MODEL EQUATIONS
        =====================

        1. NERNST POTENTIAL (reversible)
           E_rev = 1.229 - 0.85×10⁻³(T-298.15) 
                   + 4.3085×10⁻⁵·T·[ln(P_H₂) + 0.5·ln(P_O₂)]

        2. ACTIVATION LOSSES (Tafel equation)
           η_act = (R·T)/(α·F) × ln(i/i₀)
           where: R = 8.314 J/mol·K, F = 96485 C/mol
                  α = transfer coefficient (0.3-0.7)
                  i₀ = exchange current density

        3. OHMIC LOSSES
           η_ohm = i × R_total
           R_total = R_membrane + R_contacts
           R_membrane = thickness / σ(T)
           σ(T) = σ₀ × exp[0.034×(T-298.15)]

        4. CONCENTRATION LOSSES
           η_conc = B × ln[i_L/(i_L - i)]
           where: B = empirical parameter (0.01-0.1 V)
                  i_L = limiting diffusion current

        5. CELL VOLTAGE
           V_cell = E_rev - η_act - η_ohm - η_conc

        6. POWER DENSITY
           P = V_cell × i

        7. VOLTAGE EFFICIENCY
           η_eff = (V_cell / E_rev) × 100%

        VALIDITY CONDITIONS:
        • i > i₀ for η_act > 0
        • i < i_L for η_conc > 0
        • R_total > 0 for η_ohm > 0
        • Result: V(i) always decreasing (dV/di < 0)
        """
        
        text_widget = tk.Text(eq_window, wrap=tk.WORD, font=("Courier", 10),
                             bg="#f8f9fa", padx=10, pady=10)
        scrollbar = ttk.Scrollbar(eq_window, command=text_widget.yview)
        text_widget.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        text_widget.pack(fill=tk.BOTH, expand=True)
        
        text_widget.insert(tk.END, eq_text)
        text_widget.config(state=tk.DISABLED)
        
        # Close button
        ttk.Button(eq_window, text="Close", command=eq_window.destroy).pack(pady=10)
    
    def show_help(self):
        """Show help window"""
        help_window = tk.Toplevel(self.root)
        help_window.title("Help - PEMFC Modeling Tool")
        help_window.geometry("600x400")
        
        text = tk.Text(help_window, wrap=tk.WORD, font=("Arial", 10),
                      bg="#f8f9fa", padx=10, pady=10)
        scrollbar = ttk.Scrollbar(help_window, command=text.yview)
        text.configure(yscrollcommand=scrollbar.set)
        
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        text.pack(fill=tk.BOTH, expand=True)
        
        help_text = """
        PEMFC MODELING TOOL - HELP
        ==========================

        QUICK START:
        1. Adjust parameters using sliders
        2. Click '▶ Calculate Curve' to update graphs
        3. Use '📥 Export Plot' to save figures
        4. Use '📊 Export Data' to save numerical data

        PARAMETER GUIDE:
        • Temperature: 303.15-353.15 K (30-80°C)
        • Anode Pressure H2: 1.0-3.0 atm
        • Cathode Pressure O2: 1.0-3.0 atm
        • Relative Humidity: 30-100%
        • Exchange current density i₀: 1e-8 to 1e-4 A/cm²
        • Transfer coefficient α: 0.3-0.7
        • Limiting current i_L: 1.0-3.0 A/cm²
        • Ohmic resistance: 0.05-0.30 Ω·cm²
        • Concentration coefficient B: 0.01-0.10 V

        MEMBRANE PROPERTIES:
        • Nafion 211: 25 μm thickness, 0.12 S/cm conductivity
        • Nafion 112: 51 μm thickness, 0.10 S/cm conductivity
        • Nafion 117: 183 μm thickness, 0.08 S/cm conductivity

        TROUBLESHOOTING:
        • If curve looks unrealistic: Reset parameters
        • If export fails: Check write permissions
        • If interface is slow: Reduce graph resolution

        CONTACT:
        For issues or suggestions, contact the developper : amidou.maiga@2ie-edu.com.
        """
        
        text.insert(tk.END, help_text)
        text.config(state=tk.DISABLED)
        
        ttk.Button(help_window, text="Close", command=help_window.destroy).pack(pady=10)
    
    def setup_ui(self):
        """User interface Tkinter avec panneau défilable"""
        self.root = tk.Tk()
        self.root.title("PEMFC - Dynamic Modeling")
        self.root.geometry("1400x1000")  # Taille augmentée
        
        # Style
        style = ttk.Style()
        style.theme_use('clam')
        
        # Grid configuration
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_rowconfigure(3, weight=1)  # Graphiques prennent l'espace
        
        # ========== TITLE ==========
        title_frame = ttk.Frame(self.root, padding="5")
        title_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=2)
        
        title = tk.Label(title_frame, 
                        text="📊 Dynamic PEMFC Fuel Cell Modeling",
                        font=("Arial", 14, "bold"),
                        fg="#2c3e50")
        title.pack()
        
        subtitle = tk.Label(title_frame,
                           text="Interactive interface for polarization curve analysis",
                           font=("Arial", 9),
                           fg="#7f8c8d")
        subtitle.pack()
        
        # ========== SCROLLABLE INPUT PANEL ==========
        # Frame principal pour le panneau de paramètres
        input_container = ttk.Frame(self.root)
        input_container.grid(row=1, column=0, sticky="nsew", padx=5, pady=2)
        
        # Créer un Canvas avec une barre de défilement
        canvas = tk.Canvas(input_container, highlightthickness=0)
        input_container.grid_rowconfigure(0, weight=1)
        input_container.grid_columnconfigure(0, weight=1)

        canvas = tk.Canvas(input_container, height=200)  # Hauteur fixe pour le canvas
        scrollbar = ttk.Scrollbar(input_container, orient="vertical", command=canvas.yview)
        
        # Frame intérieur pour les paramètres
        self.input_frame = ttk.Frame(canvas)
        
        # Configurer le canvas
        self.input_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        self.canvas_window = canvas.create_window(
                (0, 0),
                window=self.input_frame,
                anchor="nw"
            )

        canvas.bind(
                "<Configure>",
                lambda e: canvas.itemconfig(self.canvas_window, width=e.width)
            )

        # Pack le canvas et la scrollbar
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Titre du panneau de paramètres
        ttk.Label(self.input_frame, text="🎛️ Input Parameters", 
                 font=("Arial", 11, "bold")).pack(anchor="w", pady=(0, 10))
        
        # ========== PARAMÈTRES EN 3 COLONNES ==========
        # Frame pour les colonnes
        columns_frame = ttk.Frame(self.input_frame)
        columns_frame.pack(fill="x", padx=5, pady=5)
        
        # Configure 3 colonnes
        columns_frame.grid_columnconfigure(0, weight=1)
        columns_frame.grid_columnconfigure(1, weight=1)
        columns_frame.grid_columnconfigure(2, weight=1)
        
        # === Column 1: Operating Conditions ===
        col1 = ttk.LabelFrame(columns_frame, text="🌡️ Operating Conditions", padding="10")
        col1.grid(row=0, column=0, sticky="nsew", padx=5)
        
        # Variables pour les labels
        self.temp_value = tk.StringVar(value=f"{self.default_params['T_K']:.1f} K")
        self.ph2_value = tk.StringVar(value=f"{self.default_params['P_H2']} atm")
        self.po2_value = tk.StringVar(value=f"{self.default_params['P_O2']} atm")
        self.rh_value = tk.StringVar(value=f"{self.default_params['RH']}%")
        
        # Temperature in Kelvin
        temp_frame = ttk.Frame(col1)
        temp_frame.pack(fill="x", pady=2)
        tk.Label(temp_frame, text="Temp (K):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.temp_slider = ttk.Scale(temp_frame, from_=303.15, to=353.15, orient="horizontal", length=180)
        self.temp_slider.set(self.default_params['T_K'])
        self.temp_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.temp_label = tk.Label(temp_frame, textvariable=self.temp_value, width=8, font=("Arial", 9))
        self.temp_label.pack(side="left")
        self.temp_slider.configure(command=lambda val: self.temp_value.set(f"{float(val):.1f} K"))
        
        # Anode Pressure H2
        ph2_frame = ttk.Frame(col1)
        ph2_frame.pack(fill="x", pady=2)
        tk.Label(ph2_frame, text="P H₂ (atm):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.ph2_slider = ttk.Scale(ph2_frame, from_=1.0, to=3.0, orient="horizontal", length=180)
        self.ph2_slider.set(self.default_params['P_H2'])
        self.ph2_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.ph2_label = tk.Label(ph2_frame, textvariable=self.ph2_value, width=8, font=("Arial", 9))
        self.ph2_label.pack(side="left")
        self.ph2_slider.configure(command=lambda val: self.ph2_value.set(f"{float(val):.1f} atm"))
        
        # Cathode Pressure O2
        po2_frame = ttk.Frame(col1)
        po2_frame.pack(fill="x", pady=2)
        tk.Label(po2_frame, text="P O₂ (atm):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.po2_slider = ttk.Scale(po2_frame, from_=1.0, to=3.0, orient="horizontal", length=180)
        self.po2_slider.set(self.default_params['P_O2'])
        self.po2_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.po2_label = tk.Label(po2_frame, textvariable=self.po2_value, width=8, font=("Arial", 9))
        self.po2_label.pack(side="left")
        self.po2_slider.configure(command=lambda val: self.po2_value.set(f"{float(val):.1f} atm"))
        
        # Humidity
        rh_frame = ttk.Frame(col1)
        rh_frame.pack(fill="x", pady=2)
        tk.Label(rh_frame, text="RH (%):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.rh_slider = ttk.Scale(rh_frame, from_=30, to=100, orient="horizontal", length=180)
        self.rh_slider.set(self.default_params['RH'])
        self.rh_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.rh_label = tk.Label(rh_frame, textvariable=self.rh_value, width=8, font=("Arial", 9))
        self.rh_label.pack(side="left")
        self.rh_slider.configure(command=lambda val: self.rh_value.set(f"{float(val):.0f}%"))
        
        # === Column 2: Electrochemical Parameters ===
        col2 = ttk.LabelFrame(columns_frame, text="⚡ Electrochemical Parameters", padding="10")
        col2.grid(row=0, column=1, sticky="nsew", padx=5)
        
        # Variables pour les labels
        self.alpha_value = tk.StringVar(value=f"{self.default_params['alpha']:.2f}")
        self.ilim_value = tk.StringVar(value=f"{self.default_params['i_lim']:.1f}")
        self.rohm_value = tk.StringVar(value=f"{self.default_params['R_ohm']:.2f}")
        self.b_value = tk.StringVar(value=f"{self.default_params['B']:.3f}")
        
        # Exchange current density i0
        i0_frame = ttk.Frame(col2)
        i0_frame.pack(fill="x", pady=2)
        tk.Label(i0_frame, text="i₀ (A/cm²):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.i0_var = tk.StringVar(value="1e-6")
        i0_combo = ttk.Combobox(i0_frame, textvariable=self.i0_var, width=15, font=("Arial", 9),
                               values=["1e-8", "1e-7", "1e-6", "1e-5", "1e-4"])
        i0_combo.pack(side="left", padx=5)
        
        # Transfer coefficient alpha
        alpha_frame = ttk.Frame(col2)
        alpha_frame.pack(fill="x", pady=2)
        tk.Label(alpha_frame, text="α:", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.alpha_slider = ttk.Scale(alpha_frame, from_=0.3, to=0.7, orient="horizontal", length=180)
        self.alpha_slider.set(self.default_params['alpha'])
        self.alpha_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.alpha_label = tk.Label(alpha_frame, textvariable=self.alpha_value, width=8, font=("Arial", 9))
        self.alpha_label.pack(side="left")
        self.alpha_slider.configure(command=lambda val: self.alpha_value.set(f"{float(val):.2f}"))
        
        # Limiting current i_L
        ilim_frame = ttk.Frame(col2)
        ilim_frame.pack(fill="x", pady=2)
        tk.Label(ilim_frame, text="i_L (A/cm²):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.ilim_slider = ttk.Scale(ilim_frame, from_=1.0, to=3.0, orient="horizontal", length=180)
        self.ilim_slider.set(self.default_params['i_lim'])
        self.ilim_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.ilim_label = tk.Label(ilim_frame, textvariable=self.ilim_value, width=8, font=("Arial", 9))
        self.ilim_label.pack(side="left")
        self.ilim_slider.configure(command=lambda val: self.ilim_value.set(f"{float(val):.1f}"))
        
        # Ohmic resistance
        rohm_frame = ttk.Frame(col2)
        rohm_frame.pack(fill="x", pady=2)
        tk.Label(rohm_frame, text="R_ohm (Ω·cm²):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.rohm_slider = ttk.Scale(rohm_frame, from_=0.05, to=0.3, orient="horizontal", length=180)
        self.rohm_slider.set(self.default_params['R_ohm'])
        self.rohm_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.rohm_label = tk.Label(rohm_frame, textvariable=self.rohm_value, width=8, font=("Arial", 9))
        self.rohm_label.pack(side="left")
        self.rohm_slider.configure(command=lambda val: self.rohm_value.set(f"{float(val):.2f}"))
        
        # Concentration coefficient B
        b_frame = ttk.Frame(col2)
        b_frame.pack(fill="x", pady=2)
        tk.Label(b_frame, text="B (V):", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.b_slider = ttk.Scale(b_frame, from_=0.01, to=0.1, orient="horizontal", length=180)
        self.b_slider.set(self.default_params['B'])
        self.b_slider.pack(side="left", fill="x", expand=True, padx=5)
        self.b_label = tk.Label(b_frame, textvariable=self.b_value, width=8, font=("Arial", 9))
        self.b_label.pack(side="left")
        self.b_slider.configure(command=lambda val: self.b_value.set(f"{float(val):.3f}"))
        
        # === Column 3: Material Properties ===
        col3 = ttk.LabelFrame(columns_frame, text="🔧 Material Properties", padding="10")
        col3.grid(row=0, column=2, sticky="nsew", padx=5)
        
        # Membrane type
        mem_frame = ttk.Frame(col3)
        mem_frame.pack(fill="x", pady=2)
        tk.Label(mem_frame, text="Membrane:", width=12, anchor="w", font=("Arial", 9)).pack(side="left")
        self.mem_var = tk.StringVar(value=self.default_params['membrane'])
        mem_combo = ttk.Combobox(mem_frame, textvariable=self.mem_var, width=15, font=("Arial", 9),
                                values=list(self.membrane_props.keys()))
        mem_combo.pack(side="left", padx=5)
        
        # Membrane info display
        info_frame = ttk.Frame(col3)
        info_frame.pack(fill="x", pady=10)
        
        self.mem_info_label = tk.Label(info_frame, text="", font=("Arial", 9), 
                                      fg="#7f8c8d", justify="left")
        self.mem_info_label.pack(anchor="w")
        
        # Function to update membrane info
        def update_mem_info(*args):
            mem = self.mem_var.get()
            if mem in self.membrane_props:
                props = self.membrane_props[mem]
                self.mem_info_label.config(
                    text=f"Thickness: {props['thickness']} μm\nConductivity: {props['conductivity']} S/cm"
                )
        
        self.mem_var.trace('w', update_mem_info)
        update_mem_info()  # Initial update
        
        # Espacement supplémentaire pour assurer la visibilité
        spacer = ttk.Frame(col3, height=20)
        spacer.pack()
        
        # ========== ACTION BUTTONS ==========
        button_frame = ttk.Frame(self.root, padding="5")
        button_frame.grid(row=2, column=0, sticky="ew", padx=5, pady=2)
        
        # Create all buttons in a single horizontal row
        buttons = [
            ("▶ Calculate Curve", self.update_plot),
            ("📥 Export Plot", self.export_plot),
            ("📊 Export Data", self.export_data),
            ("🔄 Reset", self.reset_params),
            ("📈 Analysis", self.show_analysis),
            ("⚙️ Equations", self.show_equations),
            ("❓ Help", self.show_help)
        ]
        
        for i, (text, command) in enumerate(buttons):
            btn = ttk.Button(button_frame, text=text, command=command, width=15)
            btn.grid(row=0, column=i, padx=2, sticky="ew")
            button_frame.grid_columnconfigure(i, weight=1)
        
        # ========== GRAPHS ==========
        graph_frame = ttk.Frame(self.root)
        graph_frame.grid(row=3, column=0, sticky="nsew", padx=5, pady=5)
        self.root.grid_rowconfigure(3, weight=1)
        
        # Create publication-quality figures
        self.fig = plt.figure(figsize=(14, 8), dpi=100)
        self.fig.patch.set_facecolor('#f8f9fa')
        
        # Subplot grid
        gs = self.fig.add_gridspec(2, 3, hspace=0.3, wspace=0.25)
        
        # 1. Main polarization curve
        self.ax1 = self.fig.add_subplot(gs[0, :2])
        self.ax1.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
        self.ax1.set_ylabel('Cell Voltage (V)', fontsize=11, fontweight='bold')
        self.ax1.set_title('PEMFC Polarization Curve', fontsize=12, fontweight='bold', pad=15)
        self.ax1.grid(True, alpha=0.2, linestyle='--')
        self.ax1.set_facecolor('#ffffff')
        
        # 2. Power curve
        self.ax2 = self.fig.add_subplot(gs[0, 2])
        self.ax2.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
        self.ax2.set_ylabel('Power (W/cm²)', fontsize=11, fontweight='bold')
        self.ax2.set_title('Power Density', fontsize=12, fontweight='bold', pad=15)
        self.ax2.grid(True, alpha=0.2, linestyle='--')
        self.ax2.set_facecolor('#ffffff')
        
        # 3. Loss distribution
        self.ax3 = self.fig.add_subplot(gs[1, 0])
        self.ax3.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
        self.ax3.set_ylabel('Overpotential (V)', fontsize=11, fontweight='bold')
        self.ax3.set_title('Loss Distribution', fontsize=12, fontweight='bold', pad=15)
        self.ax3.grid(True, alpha=0.2, linestyle='--')
        self.ax3.set_facecolor('#ffffff')
        
        # 4. Efficiency
        self.ax4 = self.fig.add_subplot(gs[1, 1])
        self.ax4.set_xlabel('Current Density (A/cm²)', fontsize=11, fontweight='bold')
        self.ax4.set_ylabel('Efficiency (%)', fontsize=11, fontweight='bold')
        self.ax4.set_title('Voltage Efficiency', fontsize=12, fontweight='bold', pad=15)
        self.ax4.grid(True, alpha=0.2, linestyle='--')
        self.ax4.set_facecolor('#ffffff')
        
        # 5. Information panel
        self.ax5 = self.fig.add_subplot(gs[1, 2])
        self.ax5.axis('off')
        
        # Tkinter integration
        self.canvas = FigureCanvasTkAgg(self.fig, master=graph_frame)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(fill="both", expand=True)
        
        # ========== STATUS BAR ==========
        status_frame = ttk.Frame(self.root, relief="sunken", padding="5")
        status_frame.grid(row=4, column=0, sticky="ew", padx=5, pady=2)
        
        self.status_label = tk.Label(status_frame, 
                                    text="Ready - Adjust parameters and click 'Calculate Curve'",
                                    font=("Arial", 9),
                                    fg="#2c3e50")
        self.status_label.pack(anchor="w")
        
        # Bind mouse wheel to scroll the parameter panel
        def on_mousewheel(event):
            canvas.yview_scroll(int(-1*(event.delta/120)), "units")
        
        canvas.bind_all("<MouseWheel>", on_mousewheel)
        
        # First calculation
        self.update_plot()
        
        print("✅ Interface utilisateur configurée avec panneau défilable")
    
    def run(self):
        """Run application"""
        self.root.mainloop()

# ========== MAIN EXECUTION ==========
if __name__ == "__main__":
    print("="*70)
    print("DYNAMIC PEMFC MODEL - VERSION AVEC PANNEAU DÉFILABLE")
    print("="*70)
    print("\nNouveautés:")
    print("✅ Panneau de paramètres avec barre de défilement")
    print("✅ Tous les sliders visibles (i_L, P_O2, etc.)")
    print("✅ Navigation avec molette de souris")
    print("✅ Interface plus grande (1400x1000)")
    print("✅ Sliders plus larges pour meilleur contrôle")
    print("="*70)
    
    print("\nLancement de l'interface...")
    
    app = PEMFC_DynamicModel()
    app.run()

DYNAMIC PEMFC MODEL - VERSION AVEC PANNEAU DÉFILABLE

Nouveautés:
✅ Panneau de paramètres avec barre de défilement
✅ Tous les sliders visibles (i_L, P_O2, etc.)
✅ Navigation avec molette de souris
✅ Interface plus grande (1400x1000)
✅ Sliders plus larges pour meilleur contrôle

Lancement de l'interface...


C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:257: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig.tight_layout()
C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:258: UserWarning: Glyph 127777 (\N{THERMOMETER}) missing from current font.
  self.canvas.draw()
C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:258: UserWarning: Glyph 128168 (\N{DASH SYMBOL}) missing from current font.
  self.canvas.draw()
C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:258: UserWarning: Glyph 128167 (\N{DROPLET}) missing from current font.
  self.canvas.draw()
C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:258: UserWarning: Glyph 128300 (\N{MICROSCOPE}) missing from current font.
  self.canvas.draw()
C:\Users\user\AppData\Local\Temp\ipykernel_24880\166740005.py:258: UserWarning: Glyph 128207 (\N{STRAIGHT RULER}) missing from current font.
  self.canvas.draw()
C

✅ Interface utilisateur configurée avec panneau défilable
✅ PEMFC_DynamicModel initialisé


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 127777 (\N{THERMOMETER}) missing from current font.
  func(*args)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 128168 (\N{DASH SYMBOL}) missing from current font.
  func(*args)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 128167 (\N{DROPLET}) missing from current font.
  func(*args)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 128300 (\N{MICROSCOPE}) missing from current font.
  func(*args)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 128207 (\N{STRAIGHT RULER}) missing from current font.
  func(*args)
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py:861: UserWarning: Glyph 128267 (\N{BATTERY}) missing from current font.
  func(*args